In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve

import sys
sys.path.append("..")
from util import print_answer

## 1. Загрузите файл classification.csv.

В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

In [2]:
df = pd.read_csv("classification.csv")
df.head()

,true,pred
0,1,0
1,1,1
2,1,1
3,0,0
4,1,1


## 2. Заполните таблицу ошибок классификации.

Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. Ответ в данном вопросе — четыре числа через пробел.

In [3]:
TP = df[(df["pred"] == 1) & (df["true"] == 1)]
FP = df[(df["pred"] == 1) & (df["true"] == 0)]
FN = df[(df["pred"] == 0) & (df["true"] == 1)]
TN = df[(df["pred"] == 0) & (df["true"] == 0)]

print_answer(1, f"{len(TP)} {len(FP)} {len(FN)} {len(TN)}")

43 34 59 64


## 3. Посчитайте основные метрики качества классификатора.

* Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score

In [4]:
acc = accuracy_score(df["true"], df["pred"])

* Precision (точность) — sklearn.metrics.precision_score

In [5]:
pr = precision_score(df["true"], df["pred"])

* Recall (полнота) — sklearn.metrics.recall_score

In [6]:
rec = recall_score(df["true"], df["pred"])

* F-мера — sklearn.metrics.f1_score

In [7]:
f1 = f1_score(df["true"], df["pred"])

В качестве ответа укажите эти четыре числа через пробел.

In [8]:
print_answer(2, f"{acc:.2f} {pr:.2f} {rec:.2f} {f1:.2f}")

0.54 0.56 0.42 0.48


## 4. Имеется четыре обученных классификатора.

В файле scores.csv записаны истинные классы и значения степени принадлежности положительному классу для каждого классификатора на некоторой выборке:

* для логистической регрессии — вероятность положительного класса (колонка score_logreg),
* для SVM — отступ от разделяющей поверхности (колонка score_svm),
* для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
* для решающего дерева — доля положительных объектов в листе (колонка score_tree).

Загрузите этот файл.

In [9]:
df2 = pd.read_csv("scores.csv")
df2.head()

,true,score_logreg,score_svm,score_knn,score_tree
0,0,0.683832,0.145976,0.787063,0.500000
1,1,0.801966,0.239511,1.000000,0.833333
2,0,0.382315,-0.245701,0.000000,0.000000
3,1,0.506797,-0.137058,0.000000,0.105263
4,1,0.488781,-0.154148,0.000000,0.105263


## 5. Посчитайте площадь под ROC-кривой для каждого классификатора.

Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

In [10]:
clf_names = df2.columns[1:]
scores = pd.Series([roc_auc_score(df2["true"], df2[clf]) for clf in clf_names], index=clf_names)

print_answer(3, scores.sort_values(ascending=False).index[0])

score_logreg


## 6. Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

Чтобы получить ответ на этот вопрос, найдите все точки precision-recall-кривой с помощью функции sklearn.metrics.precision_recall_curve. Она возвращает три массива: precision, recall, thresholds. В них записаны точность и полнота при определенных порогах, указанных в массиве thresholds. Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.

In [11]:
pr_scores = []
for clf in clf_names:
    pr_curve = precision_recall_curve(df2["true"], df2[clf])
    pr_scores.append(pr_curve[0][pr_curve[1] >= 0.7].max())

print_answer(4, clf_names[np.argmax(pr_scores)])

score_tree
